In [1]:
import os
import gc
import keras
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import utils
%matplotlib inline
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
data_dir = '../input'
!ls ../input

sample_submission_v2.csv test_v2_file_mapping.csv train_v2.csv
test-jpg                 train-jpg
test-tif-v2              train-tif-v2


In [3]:
train_label = pd.read_csv(os.path.join(data_dir, 'train_v2.csv'))
labels_str = 'agriculture, artisinal_mine, bare_ground, blooming, blow_down, clear, cloudy, conventional_mine, cultivation, habitation, haze, partly_cloudy, primary, road, selective_logging, slash_burn, water'
labels = labels_str.split(', ')
label_map = {x: labels.index(x) for x in labels}

In [4]:
def tags_to_vec(tags):
    tags_list = tags.split(' ')
    vec = np.zeros(17)
    for t in tags_list:
        vec[label_map[t]] = 1
    return vec

In [5]:
train_label['y'] = train_label.tags.apply(tags_to_vec)

In [6]:
X_train = []
y_train = []
N_train_limit = 2000
i=0
for idx, row in tqdm(train_label.iterrows(), total=min(N_train_limit, train_label.shape[0])):
    image = cv2.imread(os.path.join('../input', 'train-jpg', '{}.jpg'.format(row['image_name'])))
    #image = image[:,:,::-1]
    #plt.imshow(image)
    X_train.append(image)
    y_train.append(row['y'])
    i+=1
    if i==N_train_limit:
        break

100%|█████████▉| 1995/2000 [00:05<00:00, 346.81it/s]


In [7]:
X_train = np.stack(X_train, axis=0)
y_train = np.stack(y_train, axis=0)
print(X_train.shape, y_train.shape)

(2000, 256, 256, 3) (2000, 17)


In [8]:
xtrain, xvalid, ytrain, yvalid = train_test_split(X_train, y_train, test_size = 0.2)
print(xtrain.shape, xvalid.shape, ytrain.shape, yvalid.shape)

(1600, 256, 256, 3) (400, 256, 256, 3) (1600, 17) (400, 17)


In [9]:
try:
    del X_train, y_train
except:
    pass
gc.collect()

60

In [39]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

In [31]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(17, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='binary_crossentropy')

In [32]:
xtrain = xtrain.astype('float32')
xvalid = xvalid.astype('float32')
xtrain /= 255
xvalid /= 255

In [33]:
batch_size = 32
num_classes = 17
epochs = 200
data_augmentation = True

In [40]:
# defining a set of callbacks
class f2beta(Callback):
    def __init__(self, validation_data):
        self.validation_data = validation_data
        self.maps = []

    def eval_map(self):
        x_val, y_true = self.validation_data
        y_pred = self.model.predict(x_val)
        score = fbeta_score(y_true, y_pred, beta=2, average='samples')
        return score

    def on_epoch_end(self, epoch, logs={}):
        score = self.eval_map()
        print("f2beta for epoch %d is %f" %(epoch, score))
        self.maps.append(score)

beta_score = f2beta((xvalid, yvalid))

checkpoint = ModelCheckpoint("weights.{epoch:02d}-{val_acc:.5f}.hdf5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='auto')

earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')

In [37]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(xtrain, ytrain,
              batch_size=batch_size,
              epochs=epochs,callbacks=[checkpoint, beta_score, earlystop],
              validation_data=(xvalid, yvalid),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    
    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(xtrain)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(xtrain, ytrain,
                                     batch_size=batch_size),
                        steps_per_epoch=xtrain.shape[0] // batch_size,
                        epochs=epochs,callbacks=[checkpoint, beta_score, earlystop],
                        validation_data=(xvalid, yvalid))

Using real-time data augmentation.
Epoch 1/200
 3/50 [>.............................] - ETA: 597s - loss: 0.4690

KeyboardInterrupt: 

In [38]:
# model.fit(xtrain, ytrain, epochs=10, batch_size=32, verbose=True)

In [ ]:
# train the model on the new data for a few epochs
model.fit_generator(...)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(...)